In [1]:
# change pwd to parent directory
import os

cwd = os.getcwd()
os.chdir(os.path.dirname(cwd))

In [2]:
import numpy as np
from scipy.special import expit
from src.self_exciting.hawkes_regression import SelfExcitingLogisticRegressionWithGrad as SELR

In [3]:
# Set random seed for reproducibility
rng = np.random.default_rng(seed=42)

# Parameters for synthetic data generation
n_individuals = int(10**3)  # Number of individuals
max_events_per_individual = 10 # Maximum number of events per individual
total_events = n_individuals * max_events_per_individual  # Approximate total events

# Generate synthetic data
individuals = np.repeat(np.arange(n_individuals), max_events_per_individual)
n_events_per_individual = rng.integers(1, max_events_per_individual, size=n_individuals)

In [4]:
# Generate synthetic partner change events
true_alpha, true_gamma, true_beta_c, true_delta_c, true_beta_s, true_delta_s = (
    -3,
    [0.01, -0.05, 0.03],
    0,
    0,
    0,
    0
)

In [5]:
continuous_time = []
discrete_time = []
events = []
covariates = []

# Generate synthetic data for each individual
for i, n_events in enumerate(n_events_per_individual):
    # Generate continuous and discrete times for this individual
    continuous_times = np.sort(rng.uniform(0, 10, n_events))
    discrete_times = np.arange(1, n_events + 1)

    # Generate covariates for this individual
    age = rng.uniform(20, 50, n_events).reshape(-1, 1)
    sex = rng.choice([0, 1], size=n_events).reshape(-1, 1)
    cum_partner_count = rng.choice(np.arange(1, 10), size=n_events).reshape(-1, 1)
    covariates_individual = np.hstack([age, sex, cum_partner_count])

    s = np.zeros(n_events)
    c = np.zeros(n_events)

    for j in range(1, n_events):
        discrete_diff = discrete_times[j] - discrete_times[:j]
        continuous_diff = continuous_times[j] - continuous_times[:j]
        s[j] = np.sum(np.exp(-true_delta_s * discrete_diff) * rng.choice([0, 1], size=j))
        c[j] = np.sum(np.exp(-true_delta_c * continuous_diff) * rng.choice([0, 1], size=j))

    # Define the linear predictor for this individual
    linear_pred = (
        true_alpha
        + true_beta_s * s
        + true_beta_c * c
        + np.dot(covariates_individual, true_gamma)
        + 0.02 * age.ravel()
        - 0.0005 * (age.ravel() - 35) ** 2  # Non-monotonic effect of age
    )

    # Compute probabilities and generate binary events
    probs = expit(linear_pred)
    events_individual = rng.binomial(1, probs)

    # Store this individual's data
    continuous_time.append(continuous_times)
    discrete_time.append(discrete_times)
    events.append(events_individual)
    covariates.append(covariates_individual)

In [6]:
# Combine all individuals' data
continuous_time = np.concatenate(continuous_time)
discrete_time = np.concatenate(discrete_time)
events = np.concatenate(events)

print(f'Proportion of nonzero events: {np.sum(events)/len(events)}')

covariates = np.vstack(covariates)
covariates = (covariates - covariates.mean(axis=0)) / covariates.std(axis=0)

individuals = np.repeat(np.arange(n_individuals), max_events_per_individual)[: len(continuous_time)]

# Combine continuous and discrete times into a 2xN array
times = np.vstack([continuous_time, discrete_time])

Proportion of nonzero events: 0.13272402145837472


In [7]:
# instantiate model
model = SELR(
    max_iter=int(10**4),
    time_types=['discrete', 'continuous'],
    ignore_errors=True
    )
simple_model = SELR(
    max_iter=int(10**4),
    time_types=[],
    ignore_errors=True
    )

In [8]:
model.fit(
    times_all=times,
    events_all=events,
    individuals_all=individuals,
    covariates_all=covariates
    )

Initial parameters: [-1.89847405  0.24311241  0.02236461  0.00285138  0.1         0.01
  0.1         0.01      ]
`xtol` termination condition is satisfied.
Number of iterations: 316, function evaluations: 316, CG iterations: 319, optimality: 1.32e+02, constraint violation: 0.00e+00, execution time:  4.8 s.
Parameters: -1.87294251243889, [0.24772708 0.02573962 0.00240867], -0.06186007107335802, 0.23429008759864997, -0.053197527810982734, 0.23611655839378115
Number of parameters: 8


In [9]:
# compare true and estimated parameters
true_params = [true_alpha, true_gamma, true_beta_c, true_delta_c, true_beta_s, true_delta_s]

alpha, gamma, beta_s, beta_c, delta_s, delta_c = model.load_params()
estimated_params = [alpha, gamma, beta_c, delta_c, beta_s, delta_s]

for param_par in zip(true_params, estimated_params):
    print(f'{param_par}')

(-3, -1.87294251243889)
([0.01, -0.05, 0.03], array([0.24772708, 0.02573962, 0.00240867]))
(0, 0.23429008759864997)
(0, 0.23611655839378115)
(0, -0.06186007107335802)
(0, -0.053197527810982734)


In [10]:
simple_model.fit(
    times_all=times,
    events_all=events,
    individuals_all=individuals,
    covariates_all=covariates
    )

LogisticRegression(penalty=None)
Parameters: -1.8984740510799987, [0.24311240843373497, 0.022364606526990693, 0.002851378060717386], 0, 0, 0, 0
Number of parameters: 4


In [11]:
simple_model.alpha

-1.8984740510799987

In [12]:
from src.self_exciting.hawkes_regression import likelihood_ratio_test

In [13]:
model.n_params_

8

In [14]:
simple_model.n_params_

4

In [15]:
likelihood_ratio_test(fullmodel=model, submodel=simple_model)

(False, 1.0, -14.70499852728426)

In [16]:
simple_model.nll_

1953.4505463613334

In [17]:
model.nll_

1960.8030456249755

In [18]:
from src.self_exciting.hawkes_regression import compute_gradients

In [24]:
model.gamma.transpose()

array([0.24772708, 0.02573962, 0.00240867])

In [ ]:
params = [model.alpha, model.gamma[0], model.gamma[1], model.gamma[2], model.beta_c, model.delta_c, model.beta_s, model.delta_s]

params

[-1.87294251243889,
 0.24772708298964416,
 -0.06186007107335802,
 0.23429008759864997,
 -0.053197527810982734,
 0.23611655839378115]

In [20]:
compute_gradients(
    params=[model.alpha, model.gamma, model.beta_c, model.delta_c, model.beta_s, model.delta_s],
    events_all=events,
    times_all=times,
    covariates_all=covariates,
    positive_obs_in_groups_idx=model.compute_time_series_groups_idx(
        individuals_all=individuals,
        events_all=events,
        ),
    time_types=model.time_types,
    compute_kernels=model.compute_kernels
    )

ValueError: shapes (5033,3) and (1,3) not aligned: 3 (dim 1) != 1 (dim 0)